In [1]:
%matplotlib widget

import sys
import os
import time
import math as math
from datetime import datetime
import pickle 
import numpy as np
import astropy as ast
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.table import Table, hstack, vstack, Column
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import Angle
from astropy.time import Time
import matplotlib
import matplotlib.pyplot as plt
import scipy as sp
import scipy.integrate
import scipy.constants
import scipy.special
import scipy.misc
from scipy import integrate
from scipy.optimize import curve_fit
import scipy.interpolate
from scipy.interpolate import CubicSpline
import glob2
import shutil
from IPython.display import clear_output
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import axes3d
from tqdm.notebook import tqdm as tqdm

import cv2

import Utility3_4 as ut

pi = sp.constants.pi
k= sp.constants.k
q_e = sp.constants.e

DimY = 1096
DimX = 1936

Offset = 2 #ADU Blacklevel Offset

Gain = 2.398
SigmaGain = 0.069

DisectString = '\1/'
BackSlash = DisectString[0]
ForwardSlash = '/'


In [2]:
def MakeDarkSubSumMeanAndFluxFrameWithGehrelsError(ImagesList,MeanDarkFrame,HotsArray,DimX,DimY,Gain,SigmaGain,Offset):
    #previously just known as MakeSumMeanAndFluxFrameWithGehrelsError, but the function hasn't changed with the new name.
    #HotsArray to zero hotpixels
    #note that in the error arrays the hotpixels are set to False not 0.
    SumSignalFrame = np.zeros((DimY,DimX))
    SumDarkSubSignalFrame = np.zeros((DimY,DimX))
    MeanSignalFrame = np.zeros((DimY,DimX))
    MeanFluxFrame = np.zeros((DimY,DimX))
    ErrorSumSignalFrame = np.zeros((DimY,DimX))
    ErrorMeanSignalFrame = np.zeros((DimY,DimX))
    ErrorMeanFluxFrame = np.zeros((DimY,DimX))
    
    SumExposureTime = 0
    NumberOfFrames = float(len(ImagesList))
    ZerosArray = np.zeros((DimY,DimX))
    #Converting MeanDarkFrame from ADU to e-
    MeanDarkFrame = (MeanDarkFrame-Offset)*Gain
    
    for i in range(len(ImagesList)):
    #for all images in list of images
        image_and_metadata = ImagesList[i]
        image_data = image_and_metadata[0]
        #Convert image_data from ADU to e- accouning for ADU offset
        image_data = (image_data - Offset)*Gain
        #dark subtraction
        dark_sub_image_data = image_data - MeanDarkFrame
        SumSignalFrame = SumSignalFrame + image_data
        SumDarkSubSignalFrame = SumDarkSubSignalFrame + dark_sub_image_data
        
        SumExposureTime = SumExposureTime + float(image_and_metadata[1])
        
    MeanExposureTime = (SumExposureTime/NumberOfFrames)*(10**(-3))
    
    #Making Mean and Flux Frames (Both are Dark Subtracted buy starting with SumDarkSubSignalFrame)
    MeanSignalFrame = SumDarkSubSignalFrame/NumberOfFrames
    MeanFluxFrame = SumDarkSubSignalFrame/(MeanExposureTime*NumberOfFrames)
    
    #Zeroing the signal of hotpixels
    SumSignalFrame = np.where(HotsArray > 0, 0, SumSignalFrame)
    SumDarkSubSignalFrame = np.where(HotsArray > 0, 0, SumDarkSubSignalFrame)
    MeanSignalFrame = np.where(HotsArray > 0, 0, MeanSignalFrame)
    MeanFluxFrame = np.where(HotsArray > 0, 0.0, MeanFluxFrame)
    
    #<Determining Gehrels Error Frames>--------------------------------------------
    
    #Removing the negitives from the SumSignalFrame so sqrt still works
    #Using SumSignalFrame instead of SumDarkSubSignalFrame so that the error from dark noise is baked in
    SumSignalWithNoNegitives = np.where(SumSignalFrame<0,0,SumSignalFrame)
    
    #VarSumFrame = A_Frame + B_Frame
    #ErrorSumFrame = np.sqrt(A_Frame + B_Frame) = np.sqrt(VarSumFrame)
    
    #The contribution to the variance due to the uncertainty in Gain
    A_Frame = ((SigmaGain*SumSignalWithNoNegitives)/Gain)**2
    
    #The Gehrels Variance as if there were no uncertainty in Gain
    B_Frame = (np.sqrt(SumSignalWithNoNegitives + 0.75) + 1)**2
    
    ErrorSumSignalFrame = np.sqrt(A_Frame + B_Frame)
    
    ErrorMeanSignalFrame = ErrorSumSignalFrame/NumberOfFrames
    
    #Flux = MeanPerFrameSignal/MeanExposureTime => sigma(Flux) = (1/MeanExposureTime)*sigma(MeanPerFrameSignal)
    ErrorMeanFluxFrame = ErrorMeanSignalFrame/MeanExposureTime
    
    #Setting error to false on hot pixels which should not be counted towards error or anything else
    ErrorSumSignalFrame = np.where(HotsArray > 0, False, ErrorSumSignalFrame)
    ErrorMeanSignalFrame = np.where(HotsArray > 0, False, ErrorMeanSignalFrame)
    ErrorMeanFluxFrame = np.where(HotsArray > 0, False, ErrorMeanFluxFrame)
    #</Determining Gehrels Error Frames>-------------------------------------------
    
    #Return the (dark subtracted) Sum of all frames e- signal frame, the mean of all frames e- signal frame, and the
    #Mean flux [e-/s] of all frames frame, and associated uncertainties/error frames for all.
    #All hot pixels are zeroed in the signal and flux frames, but set to false in the error frames.
    return SumDarkSubSignalFrame, ErrorSumSignalFrame, MeanSignalFrame, ErrorMeanSignalFrame, MeanFluxFrame, ErrorMeanFluxFrame


In [3]:
def MakeSumMeanAndFluxFrameWithGehrelsError(ImagesList,MeanDarkFrame,HotsArray,DimX,DimY,Gain,SigmaGain,Offset):
    #HotsArray to zero hotpixels
    #note that in the error arrays the hotpixels are set to False not 0.
    SumSignalFrame = np.zeros((DimY,DimX))
    SumDarkSubSignalFrame = np.zeros((DimY,DimX))
    MeanSignalFrame = np.zeros((DimY,DimX))
    MeanFluxFrame = np.zeros((DimY,DimX))
    ErrorSumSignalFrame = np.zeros((DimY,DimX))
    ErrorMeanSignalFrame = np.zeros((DimY,DimX))
    ErrorMeanFluxFrame = np.zeros((DimY,DimX))
    
    SumExposureTime = 0
    NumberOfFrames = float(len(ImagesList))
    ZerosArray = np.zeros((DimY,DimX))
    #Converting MeanDarkFrame from ADU to e-
    MeanDarkFrame = (MeanDarkFrame-Offset)*Gain
    
    for i in range(len(ImagesList)):
    #for all images in list of images
        image_and_metadata = ImagesList[i]
        image_data = image_and_metadata[0]
        #Convert image_data from ADU to e- accouning for ADU offset
        image_data = (image_data - Offset)*Gain
        #dark subtraction
        SumSignalFrame = SumSignalFrame + image_data

        SumExposureTime = SumExposureTime + float(image_and_metadata[1])
        
    MeanExposureTime = (SumExposureTime/NumberOfFrames)*(10**(-3))
    
    #Making Mean and Flux Frames (Both are Dark Subtracted buy starting with SumDarkSubSignalFrame)
    MeanSignalFrame = SumSignalFrame/NumberOfFrames
    MeanFluxFrame = SumSignalFrame/(MeanExposureTime*NumberOfFrames)
    
    #Zeroing the signal of hotpixels
    SumSignalFrame = np.where(HotsArray > 0, 0, SumSignalFrame)
    MeanSignalFrame = np.where(HotsArray > 0, 0, MeanSignalFrame)
    MeanFluxFrame = np.where(HotsArray > 0, 0.0, MeanFluxFrame)
    
    #<Determining Gehrels Error Frames>--------------------------------------------
    
    #Removing the negitives from the SumSignalFrame so sqrt still works
    #Using SumSignalFrame instead of SumDarkSubSignalFrame so that the error from dark noise is baked in
    SumSignalWithNoNegitives = np.where(SumSignalFrame<0,0,SumSignalFrame)
    
    #VarSumFrame = A_Frame + B_Frame
    #ErrorSumFrame = np.sqrt(A_Frame + B_Frame) = np.sqrt(VarSumFrame)
    
    #The contribution to the variance due to the uncertainty in Gain
    A_Frame = ((SigmaGain*SumSignalWithNoNegitives)/Gain)**2
    
    #The Gehrels Variance as if there were no uncertainty in Gain
    B_Frame = (np.sqrt(SumSignalWithNoNegitives + 0.75) + 1)**2
    
    ErrorSumSignalFrame = np.sqrt(A_Frame + B_Frame)
    
    ErrorMeanSignalFrame = ErrorSumSignalFrame/NumberOfFrames
    
    #Flux = MeanPerFrameSignal/MeanExposureTime => sigma(Flux) = (1/MeanExposureTime)*sigma(MeanPerFrameSignal)
    ErrorMeanFluxFrame = ErrorMeanSignalFrame/MeanExposureTime
    
    #Setting error to false on hot pixels which should not be counted towards error or anything else
    ErrorSumSignalFrame = np.where(HotsArray > 0, False, ErrorSumSignalFrame)
    ErrorMeanSignalFrame = np.where(HotsArray > 0, False, ErrorMeanSignalFrame)
    ErrorMeanFluxFrame = np.where(HotsArray > 0, False, ErrorMeanFluxFrame)
    #</Determining Gehrels Error Frames>-------------------------------------------
    
    #Return the (dark subtracted) Sum of all frames e- signal frame, the mean of all frames e- signal frame, and the
    #Mean flux [e-/s] of all frames frame, and associated uncertainties/error frames for all.
    #All hot pixels are zeroed in the signal and flux frames, but set to false in the error frames.
    return SumSignalFrame, ErrorSumSignalFrame, MeanSignalFrame, ErrorMeanSignalFrame, MeanFluxFrame, ErrorMeanFluxFrame


In [4]:
#Loading in Master Hots Array
HotsArray = np.loadtxt('SteadyStateDataSets/MasterHotsArray.csv', delimiter=',')

SetNamesList = ['X0_Y0','X0_YM1','X0_YM2','X0_YM3','X0_YP1','X0_YP2','X0_YP3','XM2_Y0','XM2_YM1','XM2_YP1','XP2_Y0','XP2_YM1','XP2_YP1','XP4_Y0']


In [5]:
#Make the Offset Dictionary
OffsetDict = {}

for SetName in SetNamesList:
    Offset = 2
    OffsetDict.update({SetName:Offset})

#This frame has an offset of 3 for some reason. MeanFrameMedian=5.0 for this mosaic position where every other position has 4.0.
OffsetDict.update({'XM2_Y0':3}) 


In [6]:
#Making a new set of frames both dark sub and not. mean and sum with error

MeanExpTimeDict = {}
NumFrameCountDict = {} # a dictionary of DimY x DimX arrays where all entries are that set's steady state frame count
FrameCountDict = {} # a dictionary where all entries are that set's steady state frame count

for SetName in tqdm(SetNamesList):
    
    SetOffset = OffsetDict[SetName]

    datadirectory = 'SteadyStateDataSets/' + SetName +"/BeamImages/"
    filepattern = '*.FTS'
    FileNamesList = glob2.glob(datadirectory + filepattern)
    ImagesList = ut.MakeImagesList(FileNamesList,1)

    DarkFrameName = 'MeanCS_Dark'
    LoadDirectory = 'SteadyStateDataSets/' + SetName + '/'
    MeanDarkFrame = np.loadtxt(LoadDirectory + DarkFrameName + '.csv', delimiter=',') #in units of ADU
    
    #<Making Non-Dark Subtracted SumSignal, MeanSignal, and MeanFlux frames>-----------------------------------------------
    SumSignalFrame, ErrorSumSignalFrame, MeanSignalFrame, ErrorMeanSignalFrame, MeanFluxFrame, ErrorMeanFluxFrame = MakeSumMeanAndFluxFrameWithGehrelsError(ImagesList,
                                                                                                  MeanDarkFrame,HotsArray,DimX,
                                                                                                  DimY,Gain,SigmaGain,SetOffset)
    
    np.savetxt(LoadDirectory + 'PrelimNoDarkSubSumSignalFrame4' + '.csv', SumSignalFrame, delimiter=',')
    np.savetxt(LoadDirectory + 'PrelimNoDarkSubErrorSumSignalFrame4' + '.csv', ErrorSumSignalFrame, delimiter=',')
    
    np.savetxt(LoadDirectory + 'PrelimNoDarkSubMeanSignalFrame4' + '.csv', MeanSignalFrame, delimiter=',')
    np.savetxt(LoadDirectory + 'PrelimNoDarkSubErrorMeanSignalFrame4' + '.csv', ErrorMeanSignalFrame, delimiter=',')
    
    np.savetxt(LoadDirectory + 'PrelimNoDarkSubMeanFluxFrame4' + '.csv', MeanFluxFrame, delimiter=',')
    np.savetxt(LoadDirectory + 'PrelimNoDarkSubErrorMeanFluxFrame4' + '.csv', ErrorMeanFluxFrame, delimiter=',')
    #</Making Non-Dark Subtracted SumSignal, MeanSignal, and MeanFlux frames>----------------------------------------------
    
    #<Making Dark Subtracted SumSignal, MeanSignal, and MeanFlux frames>---------------------------------------------------
    SumDarkSubSignalFrame, ErrorSumSignalFrame, MeanSignalFrame, ErrorMeanSignalFrame, MeanFluxFrame, ErrorMeanFluxFrame = MakeDarkSubSumMeanAndFluxFrameWithGehrelsError(ImagesList,
                                                                                                  MeanDarkFrame,HotsArray,DimX,
                                                                                                  DimY,Gain,SigmaGain,SetOffset)
    
    np.savetxt(LoadDirectory + 'PrelimDarkSubSumSignalFrame4' + '.csv', SumDarkSubSignalFrame, delimiter=',')
    np.savetxt(LoadDirectory + 'PrelimDarkSubErrorSumSignalFrame4' + '.csv', ErrorSumSignalFrame, delimiter=',')
    
    np.savetxt(LoadDirectory + 'PrelimDarkSubMeanSignalFrame4' + '.csv', MeanSignalFrame, delimiter=',')
    np.savetxt(LoadDirectory + 'PrelimDarkSubErrorMeanSignalFrame4' + '.csv', ErrorMeanSignalFrame, delimiter=',')
    
    np.savetxt(LoadDirectory + 'PrelimDarkSubMeanFluxFrame4' + '.csv', MeanFluxFrame, delimiter=',')
    np.savetxt(LoadDirectory + 'PrelimDarkSubErrorMeanFluxFrame4' + '.csv', ErrorMeanFluxFrame, delimiter=',')
    #</Making Dark Subtracted SumSignal, MeanSignal, and MeanFlux frames>--------------------------------------------------
    
    #<Making MeanExpTimeDict>----------------------------------------------------------------------------------------------
    NumberOfFrames = float(len(ImagesList))
    SumExposureTime = 0
    
    for i in range(len(ImagesList)):
    #for all images in list of images
        image_and_metadata = ImagesList[i]
        SumExposureTime = SumExposureTime + float(image_and_metadata[1])
        
    MeanExposureTime = (SumExposureTime/NumberOfFrames)*(10**(-3)) #in units of seconds now
    
    MeanExpTimeDict.update({SetName:MeanExposureTime})
    with open('MeanExpTimeDict.pkl', 'wb') as f:
        pickle.dump(MeanExpTimeDict, f)
    #</Making MeanExpTimeDict>---------------------------------------------------------------------------------------------
    
    #<Making NumFrameCountDict and FrameCountDict>-------------------------------------------------------------------------
    NumFrameCount = len(ImagesList)
    NumFrameCountFrame = np.zeros((DimY,DimX)) + NumFrameCount
    
    NumFrameCountDict.update({SetName:NumFrameCountFrame})
    FrameCountDict.update({SetName:NumFrameCount})
    
    with open('NumFrameCountDict.pkl', 'wb') as f:
        pickle.dump(NumFrameCountDict, f)

    with open('FrameCountDict.pkl', 'wb') as f:
        pickle.dump(FrameCountDict, f)
    #</Making NumFrameCountDict and FrameCountDict>------------------------------------------------------------------------
    

  0%|          | 0/14 [00:00<?, ?it/s]

KeyboardInterrupt: 